![https://www.pngmart.com/files/5/Spaghetti-Transparent-Background.png](https://www.pngmart.com/files/5/Spaghetti-Transparent-Background.png)

In [ ]:
import pandas as pd
from math import *
import numpy as np
from scipy import interpolate

# Импорт компосовской херни
import pythoncom
from win32com.client import Dispatch, gencache

import LDefin2D
import MiscellaneousHelpers as MH



In [ ]:
#  Подключим константы API Компас
kompas6_constants = gencache.EnsureModule("{75C9F5D0-B5B8-4526-8681-9903C567D2ED}", 0, 1, 0).constants
kompas6_constants_3d = gencache.EnsureModule("{2CAF168C-7961-4B90-9DA2-701419BEEFE3}", 0, 1, 0).constants

#  Подключим описание интерфейсов API5
kompas6_api5_module = gencache.EnsureModule("{0422828C-F174-495E-AC5D-D31014DBBE87}", 0, 1, 0)
kompas_object = kompas6_api5_module.KompasObject(
    Dispatch("Kompas.Application.5")._oleobj_.QueryInterface(kompas6_api5_module.KompasObject.CLSID,
                                                             pythoncom.IID_IDispatch))
MH.iKompasObject = kompas_object

#  Подключим описание интерфейсов API7
kompas_api7_module = gencache.EnsureModule("{69AC2981-37C0-4379-84FD-5DD2F3C0A520}", 0, 1, 0)
application = kompas_api7_module.IApplication(
    Dispatch("Kompas.Application.7")._oleobj_.QueryInterface(kompas_api7_module.IApplication.CLSID,
                                                             pythoncom.IID_IDispatch))
MH.iApplication = application

documents = application.Documents
#  Получим активный документ
kompas_document = application.ActiveDocument
kompas_document_2d = kompas_api7_module.IKompasDocument2D(kompas_document)
iDocument2D = kompas_object.ActiveDocument2D()

In [ ]:
table_79 = pd.read_excel('Геодезия.xlsm', sheet_name='pandasPlan79')

table_79

In [ ]:
table_10 = pd.read_excel('Геодезия.xlsm', sheet_name='pandasPlan10')

table_10

In [ ]:
def add_layer(id: int, active: bool = True, name: str = None):
    obj = iDocument2D.ksLayer(id)
    iLayerParam = kompas6_api5_module.ksLayerParam(kompas_object.GetParamStruct(kompas6_constants.ko_LayerParam))
    iLayerParam.Init()
    if name:
        iLayerParam.name = name
    iLayerParam.state = int(active)
    iDocument2D.ksSetObjParam(obj, iLayerParam, LDefin2D.ALLPARAM)


def add_view(name: str, scale: float = 1, x=0, y=0):
    iViewParam = kompas6_api5_module.ksViewParam(kompas_object.GetParamStruct(kompas6_constants.ko_ViewParam))
    iViewParam.Init()
    iViewParam.angle = 0
    iViewParam.color = 0
    iViewParam.name = name
    iViewParam.scale_ = scale
    iViewParam.state = 3
    iViewParam.x = x
    iViewParam.y = y
    iDocument2D.ksCreateSheetView(iViewParam, 0)


def add_text(text: str, x: float, y: float, angle: float = 0, font_size: float = 2.5, color = 0):
    iParagraphParam = kompas6_api5_module.ksParagraphParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_ParagraphParam))
    iParagraphParam.Init()
    iParagraphParam.x = x
    iParagraphParam.y = y
    iParagraphParam.ang = angle
    iParagraphParam.height = 3.55
    iParagraphParam.width = 4
    iParagraphParam.hFormat = 0
    iParagraphParam.vFormat = 0
    iParagraphParam.style = 1
    iDocument2D.ksParagraph(iParagraphParam)

    iTextLineParam = kompas6_api5_module.ksTextLineParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_TextLineParam))
    iTextLineParam.Init()
    iTextLineParam.style = 1
    iTextItemArray = kompas_object.GetDynamicArray(LDefin2D.TEXT_ITEM_ARR)
    iTextItemParam = kompas6_api5_module.ksTextItemParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_TextItemParam))
    iTextItemParam.Init()
    iTextItemParam.iSNumb = 0
    iTextItemParam.s = text
    iTextItemParam.type = 0
    iTextItemFont = kompas6_api5_module.ksTextItemFont(iTextItemParam.GetItemFont())
    iTextItemFont.Init()
    iTextItemFont.bitVector = 4096
    iTextItemFont.color = color
    iTextItemFont.fontName = "GOST type A"
    iTextItemFont.height = font_size
    iTextItemFont.ksu = 1
    iTextItemArray.ksAddArrayItem(-1, iTextItemParam)
    iTextLineParam.SetTextItemArr(iTextItemArray)

    iDocument2D.ksTextLine(iTextLineParam)
    obj = iDocument2D.ksEndObj()



In [ ]:
ONE_TO_SCALE = 2000

add_view("План", 1/ONE_TO_SCALE)
add_layer(1,True,"Основные Точки")

for i, row in table_79.iterrows():
    # X и Y перевёрнуты
    obj = iDocument2D.ksCircle(row["Y"] * 1000, row["X"] * 1000, 1000, 1)
    add_text(str(row["№№"]) + " (" + str("{:1.2f}".format(row["H"])).replace(".", ",") + ")", row["Y"] * 1000 - 15000, row["X"] * 1000 - 7000)


In [ ]:
# https://stackoverflow.com/questions/7586063/how-to-calculate-the-angle-between-a-line-and-the-horizontal-axis
def angle_trunc(a):
    while a < 0.0:
        a += pi * 2
    return a

def getAngleBetweenPoints(x_orig, y_orig, x_landmark, y_landmark):
    deltaY = y_landmark - y_orig
    deltaX = x_landmark - x_orig
    return angle_trunc(atan2(deltaY, deltaX))

In [ ]:
table_10

In [ ]:
add_layer(2,True,"Побочные Точки")

extra_points = {}

for i, row in table_10.iterrows():
    if " - " in str(row["№№"]):
        origin_label, dest_label = row["№№"].split(" - ")

        if origin_label.isdigit():
            origin_label = int(origin_label)

        if dest_label.isdigit():
            dest_label = int(dest_label)

        origin = (table_79.loc[table_79['№№'] == origin_label]["Y"].values[0], table_79.loc[table_79['№№'] == origin_label]["X"].values[0])

        _dest = (table_79.loc[table_79['№№'] == dest_label]["Y"].values[0], table_79.loc[table_79['№№'] == dest_label]["X"].values[0])

        angle = np.rad2deg(getAngleBetweenPoints( origin[0], origin[1], _dest[0], _dest[1]))
        _i_am_shit_at_coding = list(origin)
        _i_am_shit_at_coding.append(table_79.loc[table_79['№№'] == origin_label]["H"].values[0])
        extra_points.update({str(origin_label): _i_am_shit_at_coding})
        continue
    else:
        if not origin:
            continue
    _d = row["Отсчет по дальномеру kl, м"]
    _alpha = row["Отсчет по гориз. Кругу"]
    xx = origin[0] + (_d * cos(np.radians(angle - _alpha)))
    yy = origin[1] + (_d * sin(np.radians(angle - _alpha)))
    obj = iDocument2D.ksLineSeg(origin[0] * 1000, origin[1] * 1000, xx * 1000,yy * 1000, 6)

    add_text(str(row["№№"]) + " (" + str("{:1.3f}".format(row["Отметки реечн. точек Hр.т., м"])).replace(".", ",") + ")", xx * 1000 - 10000, yy * 1000 - 8000)


    extra_points.update({str(row["№№"]): [xx, yy, row["Отметки реечн. точек Hр.т., м"]]})



In [ ]:
interpolated_points = {}

In [ ]:
# CON_P41 = "1,2,4,5,21".split(",")
# CON_1 = ["21"]
# CON_5 =["4"]

add_layer(3,True,"Точки интерполяции")

CON_111 = "5,6,7,8,9,112,ПЗ41".split(",")

center_point = np.array(extra_points["111"])
for point in CON_111:
    point = np.array(extra_points[point])

    if center_point[2] > point[2]:
        _coordinate_dif = center_point[:2] - point[:2]
        xia = [center_point[2], point[2]]
        hight_dif = range(int(ceil(point[2])), int(round(center_point[2])))
    else:
        _coordinate_dif = point[:2] - center_point[:2]
        xia = [point[2], center_point[2]]
        hight_dif = range(int(ceil(center_point[2])), int(round(point[2])))

    length = round(np.sqrt(np.sum(np.power(_coordinate_dif,2))))
    yia = [0, length]

    _alpha = np.rad2deg(getAngleBetweenPoints( center_point[0], center_point[1], point[0], point[1]))

    x = [0, length]
    f = interpolate.interp1d(xia, yia)

    for i in hight_dif:
        _d = f(i)
        xx = center_point[0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
        yy = center_point[1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
        obj = iDocument2D.ksPoint(xx  ,yy , 0)
        add_text(i, xx - 10000, yy - 8000, color=9410565)

        if i in interpolated_points:
            interpolated_points[i].append((xx, yy))
        else:
            interpolated_points.update({i: [(xx, yy)]})


In [ ]:
x = [extra_points[p][0] for p in extra_points]
y = [extra_points[p][1] for p in extra_points]
centroid = (sum(x) / len(extra_points), sum(y) / len(extra_points))
obj = iDocument2D.ksPoint(*(cord * 1000 for cord in centroid), 0)

current_view = kompas_document_2d.ViewsAndLayersManager.Views.ActiveView
current_view.X = 594/2 - centroid[0] / 2
current_view.Y = 420/2 - centroid[1] / 2
current_view.Update()


In [ ]:
def interpolate_line(first_point: str, second_point: str, interpolated_points: dict):

    _first_point = np.array(extra_points[str(first_point)])
    _second_point = np.array(extra_points[str(second_point)])

    _coordinate_dif = _first_point[:2] - _second_point[:2]

    _length = round(np.sqrt(np.sum(np.power(_coordinate_dif, 2))))

    _yia = [0, _length]
    _xia = [ _first_point[2], _second_point[2]]

    _x = [0, _length]
    _f = interpolate.interp1d(_xia, _yia)

    _alpha = np.rad2deg(getAngleBetweenPoints(_first_point[0], _first_point[1], _second_point[0], _second_point[1]))

    # if np.diff(sorted([int(round(_second_point[2])), int(ceil(_first_point[2]))])) <= 1:
    #     return

    _hight_dif_sorted = sorted([_second_point[2], _first_point[2]])

    _height_dif = range(int(ceil(_hight_dif_sorted[0])), int(_hight_dif_sorted[1]) + 1)

    print(f"Points: {first_point, second_point}\n Range: {_height_dif}\n {_hight_dif_sorted}\n")

    for i in _height_dif:
        _d = _f(i)
        xx = _first_point[0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
        yy = _first_point[1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
        obj = iDocument2D.ksPoint(xx, yy, 0)
        add_text(f"{i}", xx - 10000, yy - 8000, color=14417715)

        if i in interpolated_points:
            interpolated_points[i].append((xx, yy))
        else:
            interpolated_points.update({i: [(xx, yy)]})
        print(f"{i}: {xx}, {yy}")
    print("\n")

    return interpolated_points

In [ ]:
# Интерполяция между 7 и 8 точками

# _seventh = np.array(extra_points["7"])
# _eighth = np.array(extra_points["8"])
#
# _coordinate_dif = _seventh[:2] - _eighth[:2]
#
# _length = round(np.sqrt(np.sum(np.power(_coordinate_dif,2))))
#
# _yia = [0, _length]
# _xia = [_seventh[2], _eighth[2]]
#
# _x = [0, _length]
# _f = interpolate.interp1d(_xia, _yia)
#
# _alpha = np.rad2deg(getAngleBetweenPoints( _seventh[0], _seventh[1], _eighth[0], _eighth[1]))
#
# _height_dif = range(int(round(_eighth[2])), int(ceil(_seventh[2])))
#
# print(_height_dif)
#
# for i in _height_dif:
#     print(i)
#     _d = _f(i)
#     xx = _seventh[0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
#     yy = _seventh[1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
#     obj = iDocument2D.ksPoint(xx  ,yy , 0)
#     add_text(f"{i}", xx - 10000, yy - 8000, color=16711680)
#
#     if i in interpolated_points:
#         interpolated_points[i].append((xx, yy))
#     else:
#         interpolated_points.update({i: [(xx, yy)]})

interpolate_line("7", "8", interpolated_points)


In [ ]:
add_layer(4,True,"Кривые интерполяции пашни")

_max_num_of_points = max([len(interpolated_points[height_points]) for height_points in interpolated_points])

for height in interpolated_points:

    if len(interpolated_points[height]) <= 3:
        continue

    cent=(sum([p[0] for p in interpolated_points[height]])/len(interpolated_points[height]),sum([p[1] for p in interpolated_points[height]])/len(interpolated_points[height]))
    interpolated_points[height].sort( key=lambda p: atan2(p[1]-cent[1],p[0]-cent[0]))

    obj = iDocument2D.ksBezier(len(interpolated_points[height]) == _max_num_of_points, 1)
    for point in interpolated_points[height]:
        dot = iDocument2D.ksPoint(*point, 0)
    iDocument2D.ksEndObj()

In [ ]:
add_layer(5,True,"Точки интерполяции берега")

interpolated_points_shore = {}

# ПЗ 41
interpolate_line("4", "5", interpolated_points_shore)
interpolate_line("ПЗ41", "1", interpolated_points_shore)
interpolate_line("ПЗ41", "2", interpolated_points_shore)
interpolate_line("ПЗ41", "4", interpolated_points_shore)
interpolate_line("ПЗ41", "5", interpolated_points_shore)
interpolate_line("ПЗ41", "21", interpolated_points_shore)

# 112
# interpolate_line("8", "10", interpolated_points_shore)
# interpolate_line("112", "10", interpolated_points_shore)
# interpolate_line("112", "13", interpolated_points_shore)
# interpolate_line("9", "12", interpolated_points_shore)
interpolate_line("12", "13", interpolated_points_shore)
# interpolate_line("13", "1", interpolated_points_shore)
interpolate_line("1", "112", interpolated_points_shore)

# 113
interpolate_line("10", "14", interpolated_points_shore)
# interpolate_line("113", "15", interpolated_points_shore)
# interpolate_line("17", "113", interpolated_points_shore)
interpolate_line("16", "113", interpolated_points_shore)
# interpolate_line("17", "112", interpolated_points_shore)
# interpolate_line("17", "14", interpolated_points_shore)

# ПЗ 42
interpolate_line("ПЗ42", "20", interpolated_points_shore)
interpolate_line("ПЗ42", "18", interpolated_points_shore)
interpolate_line("23", "22", interpolated_points_shore)
interpolate_line("22", "21", interpolated_points_shore)
interpolate_line("20", "21", interpolated_points_shore)
interpolate_line("1", "21", interpolated_points_shore)
interpolate_line("21", "21", interpolated_points_shore)


In [ ]:
add_layer(6,True,"Кривые интерполяции берега")

for height in interpolated_points_shore:

    if len(interpolated_points_shore[height]) < 3:
        continue

    interpolated_points_shore[height].sort()

    obj = iDocument2D.ksBezier(False, 1)
    for point in interpolated_points_shore[height]:
        dot = iDocument2D.ksPoint(*point, 0)
    iDocument2D.ksEndObj()

In [ ]:
# Горизонталь вокруг ФС
add_layer(7,True,"Точки интерполяции ФС")

interpolated_points_fruit_garden = {}

_interpolation_21_23 = interpolate_line("21", "23", {})
_last_point_between_21_23 = list(_interpolation_21_23.keys())[-1]

interpolated_points_fruit_garden.update({_last_point_between_21_23: _interpolation_21_23[_last_point_between_21_23]})

_interpolation_112_17 = interpolate_line("112", "17", {})
if _last_point_between_21_23 in _interpolation_112_17:
    interpolated_points_fruit_garden[_last_point_between_21_23].append(*_interpolation_112_17[_last_point_between_21_23])

_interpolation_14_17 = interpolate_line("14", "17", {})
interpolated_points_fruit_garden[_last_point_between_21_23].append(*_interpolation_14_17[_last_point_between_21_23])

_interpolation_15_113 = interpolate_line("15", "113", {})
interpolated_points_fruit_garden[_last_point_between_21_23].append(*_interpolation_15_113[_last_point_between_21_23])

_interpolation_p42_18 = interpolate_line("ПЗ42", "18", {})
interpolated_points_fruit_garden[_last_point_between_21_23].insert(0, *_interpolation_p42_18[_last_point_between_21_23])


In [ ]:
add_layer(8,True,"Кривые интерполяции ФС")

obj = iDocument2D.ksBezier(False, 1)
for point in interpolated_points_fruit_garden[_last_point_between_21_23]:
    dot = iDocument2D.ksPoint(*point, 0)
iDocument2D.ksEndObj()

In [ ]:
# Горизонталь за ФС
add_layer(9,True,"Точки интерполяции за ФС")

interpolated_points_behind_fruit_garden = {}

_interpolation_112_10 = interpolate_line("112", "10", {})
_last_point_between_112_10 = list(_interpolation_112_10.keys())[-1]

interpolated_points_behind_fruit_garden.update({_last_point_between_112_10: _interpolation_112_10[_last_point_between_112_10]})

_interpolation_14_17 = interpolate_line("14", "17", {})
interpolated_points_behind_fruit_garden[_last_point_between_112_10].append(*_interpolation_14_17[_last_point_between_112_10])

_interpolation_15_113 = interpolate_line("15", "113", {})
if _last_point_between_112_10 in _interpolation_15_113:
    interpolated_points_behind_fruit_garden[_last_point_between_112_10].append(*_interpolation_15_113[_last_point_between_112_10])

_interpolation_8_10 = interpolate_line("8", "10", {})
interpolated_points_behind_fruit_garden[_last_point_between_112_10].insert(0, *_interpolation_8_10[_last_point_between_112_10])


In [ ]:
add_layer(10,True,"Кривые интерполяции за ФС")

obj = iDocument2D.ksBezier(False, 1)
for point in interpolated_points_behind_fruit_garden[_last_point_between_112_10]:
    dot = iDocument2D.ksPoint(*point, 0)
iDocument2D.ksEndObj()

In [ ]:
# Соединить основные границы

add_layer(10,True,"Левая граница")

_left_border = "4,5,6,7".split(sep=",")

for i in range(len(_left_border)-1):
    # print(f"{ _left_border[i]}: {extra_points[ _left_border[i]]}\t{ _left_border[i+1]}: {extra_points[ _left_border[i+1]]}")
    obj = iDocument2D.ksLineSeg(*[i * 1000 for i in (*extra_points[ _left_border[i]][:2], *extra_points[ _left_border[i+1]][:2])], 4)

In [1116]:
add_layer(11,True,"Верхняя граница")

_top_border = "7,8,10,14,15".split(sep=",")

for i in range(len(_top_border)-1):
    # print(f"{ _top_border[i]}: {extra_points[ _top_border[i]]}\t{ _top_border[i+1]}: {extra_points[ _top_border[i+1]]}")
    obj = iDocument2D.ksLineSeg(*[i * 1000 for i in (*extra_points[ _top_border[i]][:2], *extra_points[ _top_border[i+1]][:2])], 1)

In [ ]:
add_layer(12,True,"Граница пашни")

_p_border = "5,ПЗ41,9,112,8".split(sep=",")

for i in range(len(_p_border)-1):
    # print(f"{ _p_border[i]}: {extra_points[ _p_border[i]]}\t{ _p_border[i+1]}: {extra_points[ _p_border[i+1]]}")
    obj = iDocument2D.ksLineSeg(*[i * 1000 for i in (*extra_points[ _p_border[i]][:2], *extra_points[ _p_border[i+1]][:2])], 4)

In [ ]:
add_layer(13,True,"Река")

_river_border = "4,2,21,20".split(sep=",")

for i in range(len(_river_border)-1):
    # print(f"{ _river_border[i]}: {extra_points[ _river_border[i]]}\t{ _river_border[i+1]}: {extra_points[ _river_border[i+1]]}")
    obj = iDocument2D.ksLineSeg(*[i * 1000 for i in (*extra_points[ _river_border[i]][:2], *extra_points[ _river_border[i+1]][:2])], 1)

# вторая сторона реки

_alpha = np.rad2deg(getAngleBetweenPoints(*extra_points["4"][:2], *extra_points["3"][:2]))

_second_shore = []
_d = 32
for i in range(len(_river_border)):
    # obj = iDocument2D.ksLineSeg(*[i * 1000 for i in tuple(map(lambda i, j: i - j, (*extra_points[ _river_border[i]][:2], *extra_points[ _river_border[i+1]][:2]), (20,20,20,20)))], 1)

    xx = extra_points[ _river_border[i]][0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
    yy = extra_points[ _river_border[i]][1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
    _second_shore.append((xx, yy))

for i in range(len(_second_shore) - 1):
    if i != len(_second_shore) - 2:
        obj = iDocument2D.ksLineSeg(*_second_shore[i], *_second_shore[i+1], 1)
    else:
        obj = iDocument2D.ksLineSeg(*_second_shore[i], *(p * 1000 for p in extra_points["19"][:2]), 1)


In [1122]:
# Автодорога
# как же я заебався

add_layer(14,True,"Автодорога")

_alpha = np.rad2deg(getAngleBetweenPoints(*extra_points["6"][:2], *extra_points["7"][:2]))

_autobahn = []
_d = 10
for i in range(len(_top_border)):

    xx = extra_points[ _top_border[i]][0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
    yy = extra_points[ _top_border[i]][1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
    _autobahn.append((xx, yy))

for i in range(len(_autobahn) - 1):
    obj = iDocument2D.ksLineSeg(*_autobahn[i], *_autobahn[i+1], 4)
